<a href="https://colab.research.google.com/github/manishkeshwani/leaf_disease_detection/blob/main/Leaf_disease_prediction_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

import keras
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img
from keras.applications.vgg19 import VGG19,preprocess_input,decode_predictions

In [2]:
train_datagen = ImageDataGenerator(zoom_range=0.5 ,shear_range=0.3 , rescale=1/255,horizontal_flip=True,preprocessing_function=preprocess_input)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [3]:
train = train_datagen.flow_from_directory(directory = "/content/drive/MyDrive/Leaf_disease_prediction_system/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train" , 
                                          target_size=(256,256),
                                          batch_size=32)
val = val_datagen.flow_from_directory(directory = "/content/drive/MyDrive/Leaf_disease_prediction_system/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid" , 
                                          target_size=(256,256),
                                          batch_size=32)

Found 70928 images belonging to 38 classes.
Found 17637 images belonging to 38 classes.


In [4]:
t_img, label = train.next()

In [5]:
t_img.shape

(32, 256, 256, 3)

In [6]:
def plotimage(img_arr , label):
    for im , l in zip(img_arr , label):
        plt.figure(figsize=(5,5))
        #plt.imshow(im)
        plt.show()
        
#plotimage(t_img[:33], label[:3])

In [7]:
#building our model

from keras.layers import Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
import keras
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [8]:
base_model = VGG19(input_shape=(256,256,3),include_top=False)

80134624/80134624 [==============================] - 1s 0us/step


In [9]:
for layer in base_model.layers:
  layer.trainable = False

In [10]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [11]:
X = Flatten()(base_model.output)
X = Dense(units = 38 , activation = 'softmax')(X)

#Creating model
model = Model(base_model.input , X)

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [13]:
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [14]:
model.compile(optimizer = 'adam',
              loss = keras.losses.categorical_crossentropy ,
              metrics =['accuracy'])


In [15]:
#early stopping
es = EarlyStopping(monitor = 'val_accuracy',min_delta=0.01,patience = 3,verbose=1)


#model checkpoint
mc = ModelCheckpoint(filepath = 'best_model.h5',
                     monitor = 'val_accuracy',
                     min_delta=0.01,
                     patience = 3,
                     verbose=1,
                     save_best_only = True)


cb = [es,mc]

In [ ]:
his = model.fit_generator(train ,
                          steps_per_epoch = 16,
                          epochs = 50,
                          verbose = 1 ,
                          callbacks = cb ,
                          validation_data = val ,
                          validation_steps = 16)

<ipython-input-16-7931aafc66ad>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  his = model.fit_generator(train ,


Epoch 1/50
16/16 [==============================] - ETA: 0s - loss: 6.0235 - accuracy: 0.0957
Epoch 1: val_accuracy improved from -inf to 0.14258, saving model to best_model.h5
16/16 [==============================] - 234s 14s/step - loss: 6.0235 - accuracy: 0.0957 - val_loss: 37.9663 - val_accuracy: 0.1426


In [ ]:
h = his.history
h.keys()

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'],c='red')
plt.title('acc vs v-acc')
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'],c='red')
plt.title('loss vs v-loss')
plt.show()

In [ ]:
# load best model

from keras.models import load_model

model = load_model("/content/best_model.h5")


In [ ]:
# acc = model.evaluate_generator(val)[1]

# print(f"the accuracy of your model is {acc*100} %")

In [ ]:
ref = dict(zip( list(train.class_indices.values()) , list(train.class_indices.keys())))

In [ ]:
def prediction(path):

  img = load_img(path,target_size =(256,256))

  i = img_to_array(img)

  im = preprocess_input(i)

  #print(im.shape)

  img = np.expand_dims(im , axis = 0)

  #print(img.shape)

  pred = np.argmax(model.predict(img))

  #print(pred)
  print(f" the image belongs to {ref[pred]}")

In [ ]:
path = "/content/drive/MyDrive/Leaf_disease_prediction_system/test/test/AppleCedarRust3.JPG"

prediction(path)